# **Modelo Predictivo de Calidad del Aire PM2.5**
## *Análisis de Series de Tiempo y Clasificación para Predicción de Niveles de Contaminación*

---

### **Introducción**

La contaminación del aire por partículas finas (PM2.5) representa uno de los principales riesgos para la salud pública a nivel mundial. Las partículas PM2.5 son partículas en suspensión con un diámetro aerodinámico menor a 2.5 micrómetros, capaces de penetrar profundamente en el sistema respiratorio e incluso alcanzar el torrente sanguíneo.

La Organización Mundial de la Salud (OMS) establece límites recomendados de exposición a PM2.5:
- **Bueno**: 0-35 µg/m³
- **Moderado**: 35-75 µg/m³
- **No saludable para grupos sensibles**: 75-115 µg/m³
- **No saludable**: 115-150 µg/m³
- **Muy no saludable**: 150-250 µg/m³
- **Peligroso**: >250 µg/m³

### **Objetivo del Estudio**

Este análisis busca desarrollar un modelo predictivo que permita **clasificar los niveles de PM2.5 para las próximas 24 horas**, utilizando:

1. **Análisis de Series de Tiempo**: Explorar patrones temporales, estacionariedad y autocorrelación
2. **Modelos ARIMA/ARMA**: Comprender la estructura temporal de los datos
3. **Clasificación con Pipelines**: Predecir categorías de calidad del aire
4. **Métricas de Evaluación**: Precision, Recall y Accuracy para evaluar el rendimiento

### **Los Datos**

El dataset utilizado contiene **43,824 observaciones horarias** de calidad del aire y variables meteorológicas, incluyendo:

| Variable | Descripción |
|----------|-------------|
| `pm2.5` | Concentración de PM2.5 (µg/m³) - **Variable objetivo** |
| `DEWP` | Punto de rocío (°C) |
| `TEMP` | Temperatura (°C) |
| `PRES` | Presión atmosférica (hPa) |
| `cbwd` | Dirección combinada del viento |
| `Iws` | Velocidad del viento acumulada (m/s) |
| `Is` | Horas acumuladas de nieve |
| `Ir` | Horas acumuladas de lluvia |

---
## **1. Configuración del Entorno e Importación de Librerías**

In [ ]:
# Instalación de librerías necesarias (si no están instaladas)
!pip install statsmodels pmdarima -q

# Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Series de tiempo
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima import auto_arima

# Machine Learning y Pipelines
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Modelos de clasificación
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Métricas
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, classification_report, confusion_matrix,
                             precision_recall_curve, roc_auc_score)

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Librerías importadas correctamente")

---
## **2. Carga y Exploración Inicial de Datos**

In [ ]:
# Cargar datos desde GitHub (reemplazar con tu URL)
# Opción 1: Desde GitHub raw
url = 'https://raw.githubusercontent.com/Oscarpeg/ExelDireccionProyectos/main/pm25%20(1).csv'

# Opción 2: Subir archivo manualmente a Colab
# from google.colab import files
# uploaded = files.upload()
# df = pd.read_csv('pm25 (1).csv')

df = pd.read_csv(url)
print(f"Dataset cargado: {df.shape[0]:,} filas × {df.shape[1]} columnas")
df.head(10)

In [ ]:
# Información general del dataset
print("="*60)
print("INFORMACIÓN GENERAL DEL DATASET")
print("="*60)
print(f"\nDimensiones: {df.shape[0]:,} registros × {df.shape[1]} variables")
print(f"\nPeríodo temporal: {df['year'].min()} - {df['year'].max()}")
print(f"\nVariables:")
for col in df.columns:
    print(f"  • {col}: {df[col].dtype}")

print("\n" + "="*60)
print("VALORES FALTANTES")
print("="*60)
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Faltantes': missing, 'Porcentaje (%)': missing_pct})
print(missing_df[missing_df['Faltantes'] > 0])

In [ ]:
# Estadísticas descriptivas
print("="*60)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("="*60)
df.describe().round(2)

---
## **3. Preprocesamiento de Datos**

In [ ]:
# Crear columna de fecha/hora
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
df = df.set_index('datetime')

# Eliminar columnas redundantes
df = df.drop(['No', 'year', 'month', 'day', 'hour'], axis=1)

print(f"Rango temporal: {df.index.min()} a {df.index.max()}")
print(f"\nNuevas dimensiones: {df.shape}")
df.head()

In [ ]:
# Manejo de valores faltantes en pm2.5
print(f"Valores NA en pm2.5 antes: {df['pm2.5'].isna().sum():,}")

# Interpolación para series de tiempo (mejor que eliminar)
df['pm2.5'] = df['pm2.5'].interpolate(method='time')

# Para valores restantes al inicio, usar forward fill
df['pm2.5'] = df['pm2.5'].fillna(method='bfill')

print(f"Valores NA en pm2.5 después: {df['pm2.5'].isna().sum()}")

In [ ]:
# Crear variable objetivo categórica para CLASIFICACIÓN
# Basado en estándares de calidad del aire

def clasificar_pm25(valor):
    """Clasifica el nivel de PM2.5 según estándares de calidad del aire"""
    if valor <= 35:
        return 'Bueno'
    elif valor <= 75:
        return 'Moderado'
    elif valor <= 150:
        return 'No_Saludable'
    else:
        return 'Peligroso'

df['pm25_categoria'] = df['pm2.5'].apply(clasificar_pm25)

# Ver distribución de categorías
print("="*60)
print("DISTRIBUCIÓN DE CATEGORÍAS DE CALIDAD DEL AIRE")
print("="*60)
categoria_counts = df['pm25_categoria'].value_counts()
categoria_pct = (categoria_counts / len(df) * 100).round(2)

for cat in ['Bueno', 'Moderado', 'No_Saludable', 'Peligroso']:
    if cat in categoria_counts.index:
        print(f"  {cat:15} : {categoria_counts[cat]:,} ({categoria_pct[cat]}%)")

In [ ]:
# Visualización de distribución de categorías
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras
colors = ['#2ecc71', '#f39c12', '#e74c3c', '#8e44ad']
orden = ['Bueno', 'Moderado', 'No_Saludable', 'Peligroso']
categoria_ordenada = df['pm25_categoria'].value_counts().reindex(orden)

axes[0].bar(categoria_ordenada.index, categoria_ordenada.values, color=colors)
axes[0].set_title('Distribución de Categorías de Calidad del Aire', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Categoría')
axes[0].set_ylabel('Frecuencia')
axes[0].tick_params(axis='x', rotation=45)

# Gráfico de pastel
axes[1].pie(categoria_ordenada.values, labels=categoria_ordenada.index, autopct='%1.1f%%', 
            colors=colors, explode=[0.02]*len(categoria_ordenada))
axes[1].set_title('Proporción de Categorías', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

---
## **4. Análisis de Series de Tiempo**

### 4.1 Visualización de la Serie Temporal

In [ ]:
# Visualizar la serie de tiempo completa
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Serie completa
axes[0].plot(df.index, df['pm2.5'], linewidth=0.5, alpha=0.7, color='steelblue')
axes[0].set_title('Serie Temporal Completa de PM2.5', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Fecha')
axes[0].set_ylabel('PM2.5 (µg/m³)')
axes[0].axhline(y=35, color='green', linestyle='--', label='Límite Bueno (35)')
axes[0].axhline(y=75, color='orange', linestyle='--', label='Límite Moderado (75)')
axes[0].axhline(y=150, color='red', linestyle='--', label='Límite No Saludable (150)')
axes[0].legend(loc='upper right')

# Muestra de un mes (para ver patrones diarios)
muestra = df['pm2.5']['2013-06':'2013-06']
axes[1].plot(muestra.index, muestra.values, linewidth=1, color='steelblue', marker='o', markersize=2)
axes[1].set_title('Detalle: Junio 2013 (Patrones Diarios)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Fecha')
axes[1].set_ylabel('PM2.5 (µg/m³)')
axes[1].axhline(y=35, color='green', linestyle='--', alpha=0.7)
axes[1].axhline(y=75, color='orange', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

### 4.2 Análisis de Estacionariedad

Una serie es **estacionaria** si sus propiedades estadísticas (media, varianza, covarianza) no cambian con el tiempo. Esto es fundamental para modelos ARMA/ARIMA.

In [ ]:
def test_estacionariedad(serie, nombre='Serie'):
    """
    Realiza el test de Dickey-Fuller Aumentado (ADF) para verificar estacionariedad.
    
    Hipótesis:
    - H0: La serie tiene raíz unitaria (NO es estacionaria)
    - H1: La serie NO tiene raíz unitaria (ES estacionaria)
    
    Si p-valor < 0.05, rechazamos H0 → Serie ES ESTACIONARIA
    """
    print("="*60)
    print(f"TEST DE DICKEY-FULLER AUMENTADO (ADF) - {nombre}")
    print("="*60)
    
    resultado = adfuller(serie.dropna(), autolag='AIC')
    
    print(f'Estadístico ADF: {resultado[0]:.4f}')
    print(f'P-valor: {resultado[1]:.6f}')
    print(f'Lags utilizados: {resultado[2]}')
    print(f'Observaciones: {resultado[3]}')
    print('\nValores críticos:')
    for key, value in resultado[4].items():
        print(f'   {key}: {value:.4f}')
    
    print("\n" + "-"*60)
    if resultado[1] < 0.05:
        print("✓ CONCLUSIÓN: La serie ES ESTACIONARIA (p-valor < 0.05)")
        print("  Se rechaza H0: la serie no tiene raíz unitaria")
        return True
    else:
        print("✗ CONCLUSIÓN: La serie NO ES ESTACIONARIA (p-valor >= 0.05)")
        print("  No se puede rechazar H0: la serie podría tener raíz unitaria")
        return False

# Test en la serie original
es_estacionaria = test_estacionariedad(df['pm2.5'], 'PM2.5 Original')

In [ ]:
# Si no es estacionaria, aplicar diferenciación
if not es_estacionaria:
    print("\nAplicando diferenciación de primer orden...\n")
    df['pm25_diff'] = df['pm2.5'].diff()
    test_estacionariedad(df['pm25_diff'].dropna(), 'PM2.5 Diferenciada (d=1)')

### 4.3 Análisis de Covarianza y Autocorrelación

La **covarianza** en series de tiempo mide cómo los valores de la serie en diferentes momentos se relacionan entre sí.

- **ACF (Autocorrelación)**: Correlación entre la serie y sus valores rezagados
- **PACF (Autocorrelación Parcial)**: Correlación directa entre observaciones separadas por k períodos

In [ ]:
# Calcular y mostrar covarianza entre PM2.5 y sus lags
print("="*60)
print("ANÁLISIS DE COVARIANZA TEMPORAL")
print("="*60)

# Crear lags
lags_covarianza = [1, 6, 12, 24, 48, 168]  # 1h, 6h, 12h, 1día, 2días, 1semana
print("\nCovarianza entre PM2.5(t) y PM2.5(t-k):")
print("-"*40)

for lag in lags_covarianza:
    cov = df['pm2.5'].cov(df['pm2.5'].shift(lag))
    corr = df['pm2.5'].corr(df['pm2.5'].shift(lag))
    if lag == 1:
        lag_desc = f"{lag} hora"
    elif lag < 24:
        lag_desc = f"{lag} horas"
    elif lag == 24:
        lag_desc = "1 día (24h)"
    elif lag == 48:
        lag_desc = "2 días (48h)"
    else:
        lag_desc = "1 semana (168h)"
    print(f"  Lag {lag:3d} ({lag_desc:12}): Cov = {cov:12.2f}, Corr = {corr:.4f}")

print("\n" + "-"*60)
print("INTERPRETACIÓN:")
print("• Covarianza alta → valores similares tienden a ocurrir juntos")
print("• Correlación cercana a 1 → fuerte dependencia temporal")
print("• Decaimiento lento → memoria larga en la serie")

In [ ]:
# Gráficos ACF y PACF
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# ACF serie original (48 lags = 2 días)
plot_acf(df['pm2.5'].dropna(), lags=48, ax=axes[0, 0], alpha=0.05)
axes[0, 0].set_title('ACF - Serie Original (48 horas)', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Lag (horas)')

# PACF serie original
plot_pacf(df['pm2.5'].dropna(), lags=48, ax=axes[0, 1], alpha=0.05)
axes[0, 1].set_title('PACF - Serie Original (48 horas)', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Lag (horas)')

# ACF serie diferenciada
if 'pm25_diff' in df.columns:
    plot_acf(df['pm25_diff'].dropna(), lags=48, ax=axes[1, 0], alpha=0.05)
    axes[1, 0].set_title('ACF - Serie Diferenciada (48 horas)', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('Lag (horas)')
    
    plot_pacf(df['pm25_diff'].dropna(), lags=48, ax=axes[1, 1], alpha=0.05)
    axes[1, 1].set_title('PACF - Serie Diferenciada (48 horas)', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Lag (horas)')

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("INTERPRETACIÓN DE ACF Y PACF")
print("="*60)
print("""
• ACF (Autocorrelación): 
  - Decaimiento lento → indica componente AR
  - Corte abrupto en lag q → indica orden MA(q)
  
• PACF (Autocorrelación Parcial):
  - Corte abrupto en lag p → indica orden AR(p)
  - Decaimiento lento → indica componente MA
  
• Para ARMA(p,q): Ambos decaen gradualmente
""")

### 4.4 Descomposición de la Serie Temporal

In [ ]:
# Descomposición de la serie (usando datos diarios para mejor visualización)
pm25_diario = df['pm2.5'].resample('D').mean()

# Descomposición aditiva
descomposicion = seasonal_decompose(pm25_diario.dropna(), model='additive', period=365)

fig, axes = plt.subplots(4, 1, figsize=(15, 12))

descomposicion.observed.plot(ax=axes[0], color='steelblue')
axes[0].set_title('Serie Observada', fontsize=12, fontweight='bold')
axes[0].set_ylabel('PM2.5')

descomposicion.trend.plot(ax=axes[1], color='orange')
axes[1].set_title('Tendencia', fontsize=12, fontweight='bold')
axes[1].set_ylabel('PM2.5')

descomposicion.seasonal.plot(ax=axes[2], color='green')
axes[2].set_title('Estacionalidad (Anual)', fontsize=12, fontweight='bold')
axes[2].set_ylabel('PM2.5')

descomposicion.resid.plot(ax=axes[3], color='red')
axes[3].set_title('Residuos', fontsize=12, fontweight='bold')
axes[3].set_ylabel('PM2.5')

plt.tight_layout()
plt.show()

print("\nCOMPONENTES DE LA SERIE:")
print("• Tendencia: Movimiento a largo plazo")
print("• Estacionalidad: Patrones que se repiten (anual)")
print("• Residuos: Variación aleatoria no explicada")

### 4.5 Modelo ARIMA/ARMA

**Conceptos clave:**
- **AR (Autoregresivo)**: El valor actual depende de valores pasados
  - $y_t = c + \phi_1 y_{t-1} + \phi_2 y_{t-2} + ... + \epsilon_t$
  
- **MA (Media Móvil)**: El valor actual depende de errores pasados
  - $y_t = c + \epsilon_t + \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} + ...$
  
- **ARMA**: Combinación de AR y MA
- **ARIMA**: ARMA con diferenciación para series no estacionarias

In [ ]:
# Usar una muestra para el modelo ARIMA (datos horarios del último año)
pm25_muestra = df['pm2.5']['2014-01-01':].copy()

print(f"Usando {len(pm25_muestra):,} observaciones para el modelo ARIMA")
print(f"Período: {pm25_muestra.index.min()} a {pm25_muestra.index.max()}")

# Auto ARIMA para encontrar los mejores parámetros
print("\nBuscando los mejores parámetros ARIMA (esto puede tomar unos minutos)...")

modelo_auto = auto_arima(
    pm25_muestra,
    start_p=0, start_q=0,
    max_p=3, max_q=3,
    d=None,  # Auto-detectar diferenciación
    seasonal=False,
    trace=True,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True
)

print("\n" + "="*60)
print("MEJOR MODELO ENCONTRADO")
print("="*60)
print(modelo_auto.summary())

In [ ]:
# Predicción con ARIMA para las próximas 24 horas
n_prediccion = 24
prediccion_arima, conf_int = modelo_auto.predict(n_periods=n_prediccion, return_conf_int=True)

# Crear índice para predicciones
ultimo_indice = pm25_muestra.index[-1]
indice_prediccion = pd.date_range(start=ultimo_indice + pd.Timedelta(hours=1), periods=n_prediccion, freq='H')

# Visualizar
fig, ax = plt.subplots(figsize=(15, 6))

# Últimas 72 horas de datos reales
ultimas_horas = pm25_muestra[-72:]
ax.plot(ultimas_horas.index, ultimas_horas.values, 'b-', label='Datos Reales', linewidth=2)

# Predicciones
ax.plot(indice_prediccion, prediccion_arima, 'r--', label='Predicción ARIMA (24h)', linewidth=2)
ax.fill_between(indice_prediccion, conf_int[:, 0], conf_int[:, 1], color='red', alpha=0.2, label='Intervalo de Confianza 95%')

ax.axhline(y=35, color='green', linestyle=':', alpha=0.7, label='Límite Bueno')
ax.axhline(y=75, color='orange', linestyle=':', alpha=0.7, label='Límite Moderado')

ax.set_title('Predicción ARIMA de PM2.5 para las Próximas 24 Horas', fontsize=14, fontweight='bold')
ax.set_xlabel('Fecha/Hora')
ax.set_ylabel('PM2.5 (µg/m³)')
ax.legend(loc='upper left')
plt.tight_layout()
plt.show()

# Mostrar predicciones
print("\n" + "="*60)
print("PREDICCIONES ARIMA PARA LAS PRÓXIMAS 24 HORAS")
print("="*60)
for i, (fecha, pred) in enumerate(zip(indice_prediccion, prediccion_arima)):
    categoria = clasificar_pm25(pred)
    print(f"Hora {i+1:2d} ({fecha.strftime('%Y-%m-%d %H:%M')}): {pred:6.1f} µg/m³ → {categoria}")

---
## **5. Modelo de Clasificación con Pipelines**

Ahora construiremos un modelo de **clasificación** para predecir la categoría de calidad del aire (Bueno, Moderado, No_Saludable, Peligroso) utilizando **Pipelines** de scikit-learn.

In [ ]:
# Preparar datos para clasificación
df_clasif = df.copy()

# Feature Engineering: agregar variables temporales y lags
df_clasif['hora'] = df_clasif.index.hour
df_clasif['dia_semana'] = df_clasif.index.dayofweek
df_clasif['mes'] = df_clasif.index.month
df_clasif['es_fin_semana'] = (df_clasif['dia_semana'] >= 5).astype(int)

# Lags de PM2.5 (valores pasados)
for lag in [1, 2, 3, 6, 12, 24]:
    df_clasif[f'pm25_lag_{lag}'] = df_clasif['pm2.5'].shift(lag)

# Media móvil
df_clasif['pm25_media_6h'] = df_clasif['pm2.5'].rolling(window=6).mean()
df_clasif['pm25_media_24h'] = df_clasif['pm2.5'].rolling(window=24).mean()

# Eliminar filas con NaN
df_clasif = df_clasif.dropna()

print(f"Datos para clasificación: {df_clasif.shape[0]:,} registros")
df_clasif.head()

In [ ]:
# Crear variable objetivo para predicción a 24 horas
# Queremos predecir la categoría de PM2.5 en 24 horas
df_clasif['target_24h'] = df_clasif['pm25_categoria'].shift(-24)
df_clasif = df_clasif.dropna()

# Definir features y target
features_numericas = ['DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir',
                      'hora', 'dia_semana', 'mes', 'es_fin_semana',
                      'pm25_lag_1', 'pm25_lag_2', 'pm25_lag_3', 
                      'pm25_lag_6', 'pm25_lag_12', 'pm25_lag_24',
                      'pm25_media_6h', 'pm25_media_24h', 'pm2.5']

features_categoricas = ['cbwd']

X = df_clasif[features_numericas + features_categoricas]
y = df_clasif['target_24h']

# Codificar target
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"Features: {X.shape[1]}")
print(f"Muestras: {X.shape[0]:,}")
print(f"\nClases: {le.classes_}")

In [ ]:
# División temporal de datos (importante para series de tiempo)
# No usar split aleatorio - mantener orden temporal

split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y_encoded[:split_idx], y_encoded[split_idx:]

print(f"Conjunto de entrenamiento: {len(X_train):,} muestras")
print(f"Conjunto de prueba: {len(X_test):,} muestras")
print(f"\nPeríodo entrenamiento: {X_train.index.min()} a {X_train.index.max()}")
print(f"Período prueba: {X_test.index.min()} a {X_test.index.max()}")

In [ ]:
# Crear Pipeline de preprocesamiento

# Preprocesador para variables numéricas
preprocessor_numerico = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocesador para variables categóricas
preprocessor_categorico = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combinar preprocesadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', preprocessor_numerico, features_numericas),
        ('cat', preprocessor_categorico, features_categoricas)
    ])

print("✓ Pipeline de preprocesamiento creado")
print("\nEstructura del preprocesador:")
print("  1. Variables numéricas: Imputación (mediana) → Escalado (StandardScaler)")
print("  2. Variables categóricas: Imputación (moda) → One-Hot Encoding")

In [ ]:
# Crear Pipelines completos con diferentes modelos

modelos = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

pipelines = {}
for nombre, modelo in modelos.items():
    pipelines[nombre] = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', modelo)
    ])

print("✓ Pipelines creados para los siguientes modelos:")
for nombre in pipelines.keys():
    print(f"  • {nombre}")

In [ ]:
# Entrenar y evaluar modelos
resultados = []

print("="*80)
print("ENTRENAMIENTO Y EVALUACIÓN DE MODELOS")
print("="*80)

for nombre, pipeline in pipelines.items():
    print(f"\nEntrenando {nombre}...")
    
    # Entrenar
    pipeline.fit(X_train, y_train)
    
    # Predecir
    y_pred = pipeline.predict(X_test)
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    resultados.append({
        'Modelo': nombre,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    })
    
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-Score:  {f1:.4f}")

# Tabla de resultados
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values('F1-Score', ascending=False)

print("\n" + "="*80)
print("COMPARACIÓN DE MODELOS")
print("="*80)
print(df_resultados.to_string(index=False))

---
## **6. Evaluación Detallada del Mejor Modelo**

In [ ]:
# Seleccionar el mejor modelo
mejor_modelo_nombre = df_resultados.iloc[0]['Modelo']
mejor_pipeline = pipelines[mejor_modelo_nombre]

print(f"Mejor modelo: {mejor_modelo_nombre}")
print("\n" + "="*60)

# Predicciones del mejor modelo
y_pred_mejor = mejor_pipeline.predict(X_test)

# Reporte de clasificación detallado
print("\nREPORTE DE CLASIFICACIÓN DETALLADO")
print("="*60)
print(classification_report(y_test, y_pred_mejor, target_names=le.classes_))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_mejor)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Matriz de confusión absoluta
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=le.classes_, yticklabels=le.classes_, ax=axes[0])
axes[0].set_title(f'Matriz de Confusión - {mejor_modelo_nombre}', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Predicción')
axes[0].set_ylabel('Valor Real')

# Matriz de confusión normalizada
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Blues', 
            xticklabels=le.classes_, yticklabels=le.classes_, ax=axes[1])
axes[1].set_title('Matriz de Confusión Normalizada', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Predicción')
axes[1].set_ylabel('Valor Real')

plt.tight_layout()
plt.show()

In [ ]:
# Explicación de métricas
print("="*70)
print("EXPLICACIÓN DE MÉTRICAS DE CLASIFICACIÓN")
print("="*70)
print("""
┌─────────────────────────────────────────────────────────────────────┐
│ ACCURACY (Exactitud)                                                │
│ ─────────────────────                                               │
│ Proporción de predicciones correctas sobre el total.                │
│ Fórmula: (VP + VN) / (VP + VN + FP + FN)                           │
│                                                                     │
│ Interpretación: ¿Qué tan frecuentemente el modelo acierta?         │
├─────────────────────────────────────────────────────────────────────┤
│ PRECISION (Precisión)                                               │
│ ────────────────────                                                │
│ De todas las predicciones positivas, ¿cuántas son correctas?       │
│ Fórmula: VP / (VP + FP)                                            │
│                                                                     │
│ Interpretación: ¿Qué tan confiables son las alertas del modelo?    │
│ Alta precisión = pocas falsas alarmas                              │
├─────────────────────────────────────────────────────────────────────┤
│ RECALL (Sensibilidad/Exhaustividad)                                 │
│ ──────────────────────────────────                                  │
│ De todos los casos positivos reales, ¿cuántos detectamos?          │
│ Fórmula: VP / (VP + FN)                                            │
│                                                                     │
│ Interpretación: ¿Qué tan bueno es detectando casos peligrosos?     │
│ Alto recall = pocos casos peligrosos sin detectar                  │
├─────────────────────────────────────────────────────────────────────┤
│ F1-SCORE                                                            │
│ ────────                                                            │
│ Media armónica de Precision y Recall.                              │
│ Fórmula: 2 × (Precision × Recall) / (Precision + Recall)           │
│                                                                     │
│ Interpretación: Balance entre precisión y exhaustividad            │
└─────────────────────────────────────────────────────────────────────┘

En el contexto de calidad del aire:
• Alta PRECISION → Menos falsas alarmas de contaminación
• Alto RECALL → Detectamos más episodios de contaminación real
• Para salud pública, RECALL es más importante (no queremos perder
  episodios peligrosos)
""")

In [ ]:
# Visualización comparativa de métricas
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras comparativo
metricas = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x = np.arange(len(metricas))
width = 0.2

for i, (_, row) in enumerate(df_resultados.iterrows()):
    valores = [row['Accuracy'], row['Precision'], row['Recall'], row['F1-Score']]
    axes[0].bar(x + i*width, valores, width, label=row['Modelo'])

axes[0].set_xlabel('Métrica')
axes[0].set_ylabel('Valor')
axes[0].set_title('Comparación de Métricas por Modelo', fontsize=12, fontweight='bold')
axes[0].set_xticks(x + width * 1.5)
axes[0].set_xticklabels(metricas)
axes[0].legend(loc='lower right')
axes[0].set_ylim(0, 1)

# Gráfico radar del mejor modelo
valores_mejor = df_resultados.iloc[0][['Accuracy', 'Precision', 'Recall', 'F1-Score']].values
categorias = ['Accuracy', 'Precision', 'Recall', 'F1-Score']

# Cerrar el gráfico radar
valores_mejor = np.concatenate((valores_mejor, [valores_mejor[0]]))
angulos = np.linspace(0, 2 * np.pi, len(categorias), endpoint=False).tolist()
angulos += angulos[:1]

ax_radar = fig.add_subplot(122, polar=True)
ax_radar.plot(angulos, valores_mejor, 'o-', linewidth=2, color='steelblue')
ax_radar.fill(angulos, valores_mejor, alpha=0.25, color='steelblue')
ax_radar.set_xticks(angulos[:-1])
ax_radar.set_xticklabels(categorias)
ax_radar.set_title(f'Perfil de Métricas - {mejor_modelo_nombre}', fontsize=12, fontweight='bold')

# Ocultar el segundo subplot original
axes[1].axis('off')

plt.tight_layout()
plt.show()

---
## **7. Validación con Series de Tiempo (TimeSeriesSplit)**

In [ ]:
# Validación cruzada específica para series de tiempo
tscv = TimeSeriesSplit(n_splits=5)

print("="*60)
print("VALIDACIÓN CRUZADA CON TIMESERIESSPLIT")
print("="*60)
print("\nEste método respeta el orden temporal de los datos,")
print("entrenando siempre con datos pasados y validando con futuros.\n")

# Preparar datos (sin índice de tiempo)
X_array = preprocessor.fit_transform(X)

for nombre, modelo in modelos.items():
    scores = cross_val_score(modelo, X_array, y_encoded, cv=tscv, scoring='accuracy')
    print(f"{nombre:20}: Accuracy = {scores.mean():.4f} (+/- {scores.std()*2:.4f})")

In [ ]:
# Visualizar los folds de TimeSeriesSplit
fig, ax = plt.subplots(figsize=(14, 4))

for i, (train_idx, test_idx) in enumerate(tscv.split(X)):
    ax.scatter(train_idx, [i+1]*len(train_idx), c='blue', marker='s', s=1, label='Train' if i==0 else '')
    ax.scatter(test_idx, [i+1]*len(test_idx), c='red', marker='s', s=1, label='Test' if i==0 else '')

ax.set_xlabel('Índice de muestra (tiempo →)')
ax.set_ylabel('Fold')
ax.set_title('Visualización de TimeSeriesSplit (5 folds)', fontsize=12, fontweight='bold')
ax.legend(loc='upper left')
ax.set_yticks([1, 2, 3, 4, 5])

plt.tight_layout()
plt.show()

print("\nNota: El conjunto de entrenamiento (azul) siempre precede al de prueba (rojo)")
print("      Esto simula predicción de eventos futuros basada en datos pasados.")

---
## **8. Predicción de las Próximas 24 Horas**

In [ ]:
# Obtener las últimas observaciones para predecir
ultimas_obs = X.iloc[-24:].copy()

# Predecir con el mejor modelo
predicciones_24h = mejor_pipeline.predict(ultimas_obs)
predicciones_labels = le.inverse_transform(predicciones_24h)

# Si el modelo soporta probabilidades
if hasattr(mejor_pipeline.named_steps['classifier'], 'predict_proba'):
    probabilidades = mejor_pipeline.predict_proba(ultimas_obs)
else:
    probabilidades = None

print("="*70)
print("PREDICCIÓN DE CALIDAD DEL AIRE - PRÓXIMAS 24 HORAS")
print(f"Modelo utilizado: {mejor_modelo_nombre}")
print("="*70)

# Crear DataFrame con predicciones
df_pred = pd.DataFrame({
    'Hora': range(1, 25),
    'Fecha_Hora': pd.date_range(start=ultimas_obs.index[-1] + pd.Timedelta(hours=1), periods=24, freq='H'),
    'Categoría_Predicha': predicciones_labels
})

if probabilidades is not None:
    for i, clase in enumerate(le.classes_):
        df_pred[f'Prob_{clase}'] = probabilidades[:, i]

print(df_pred.to_string(index=False))

In [ ]:
# Visualización de predicciones
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Colores por categoría
color_map = {'Bueno': '#2ecc71', 'Moderado': '#f39c12', 'No_Saludable': '#e74c3c', 'Peligroso': '#8e44ad'}
colores = [color_map.get(cat, 'gray') for cat in predicciones_labels]

# Gráfico de categorías predichas
axes[0].bar(range(1, 25), [1]*24, color=colores, edgecolor='black', linewidth=0.5)
axes[0].set_xlabel('Hora de predicción')
axes[0].set_title('Predicción de Categoría de Calidad del Aire - Próximas 24 Horas', 
                  fontsize=14, fontweight='bold')
axes[0].set_xticks(range(1, 25))
axes[0].set_yticks([])

# Leyenda
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=color, label=cat) for cat, color in color_map.items()]
axes[0].legend(handles=legend_elements, loc='upper right')

# Añadir texto con la categoría
for i, cat in enumerate(predicciones_labels):
    axes[0].text(i+1, 0.5, cat[:3], ha='center', va='center', fontsize=8, fontweight='bold', color='white')

# Gráfico de probabilidades (si están disponibles)
if probabilidades is not None:
    x = range(1, 25)
    bottom = np.zeros(24)
    for i, clase in enumerate(le.classes_):
        axes[1].bar(x, probabilidades[:, i], bottom=bottom, label=clase, 
                   color=color_map.get(clase, 'gray'), alpha=0.8)
        bottom += probabilidades[:, i]
    
    axes[1].set_xlabel('Hora de predicción')
    axes[1].set_ylabel('Probabilidad')
    axes[1].set_title('Distribución de Probabilidades por Hora', fontsize=14, fontweight='bold')
    axes[1].set_xticks(range(1, 25))
    axes[1].legend(loc='upper right')
    axes[1].set_ylim(0, 1)

plt.tight_layout()
plt.show()

In [ ]:
# Resumen de predicciones
print("="*60)
print("RESUMEN DE PREDICCIONES - PRÓXIMAS 24 HORAS")
print("="*60)

resumen = pd.Series(predicciones_labels).value_counts()
for cat in ['Bueno', 'Moderado', 'No_Saludable', 'Peligroso']:
    if cat in resumen.index:
        horas = resumen[cat]
        pct = horas / 24 * 100
        print(f"  {cat:15}: {horas:2d} horas ({pct:.1f}%)")

# Alertas
horas_peligrosas = sum(1 for cat in predicciones_labels if cat in ['No_Saludable', 'Peligroso'])
if horas_peligrosas > 0:
    print("\n" + "⚠"*30)
    print(f"  ALERTA: Se predicen {horas_peligrosas} horas con calidad del aire")
    print(f"          no saludable o peligrosa en las próximas 24 horas.")
    print("⚠"*30)
else:
    print("\n✓ No se predicen episodios de contaminación severa en las próximas 24 horas.")

---
## **9. Importancia de Variables**

In [ ]:
# Obtener importancia de variables (para modelos basados en árboles)
if mejor_modelo_nombre in ['Random Forest', 'Gradient Boosting']:
    # Obtener nombres de features después del preprocesamiento
    feature_names = features_numericas.copy()
    
    # Añadir nombres de features categóricas (one-hot encoded)
    if hasattr(preprocessor.named_transformers_['cat'].named_steps['onehot'], 'get_feature_names_out'):
        cat_features = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(features_categoricas)
        feature_names.extend(cat_features)
    
    importancia = mejor_pipeline.named_steps['classifier'].feature_importances_
    
    # Crear DataFrame de importancia
    df_importancia = pd.DataFrame({
        'Feature': feature_names[:len(importancia)],
        'Importancia': importancia
    }).sort_values('Importancia', ascending=True)
    
    # Visualizar
    fig, ax = plt.subplots(figsize=(10, 8))
    
    colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(df_importancia)))
    ax.barh(df_importancia['Feature'], df_importancia['Importancia'], color=colors)
    ax.set_xlabel('Importancia')
    ax.set_title(f'Importancia de Variables - {mejor_modelo_nombre}', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("\nTop 5 variables más importantes:")
    for _, row in df_importancia.tail(5).iloc[::-1].iterrows():
        print(f"  • {row['Feature']}: {row['Importancia']:.4f}")
else:
    print(f"La importancia de variables no está disponible para {mejor_modelo_nombre}")

---
## **10. Conclusiones**

### Hallazgos Principales

In [ ]:
print("="*70)
print("CONCLUSIONES DEL ANÁLISIS")
print("="*70)

print("""
1. ANÁLISIS DE SERIES DE TIEMPO
   ─────────────────────────────
   • La serie de PM2.5 presenta patrones de autocorrelación significativos,
     indicando que los valores pasados son útiles para predecir valores futuros.
   • Se identificó estacionalidad en los datos, con patrones diarios y anuales.
   • La covarianza temporal muestra dependencia fuerte en las primeras horas,
     decayendo gradualmente con el tiempo.

2. MODELOS ARIMA/ARMA
   ──────────────────
   • Se utilizó auto_arima para identificar automáticamente los mejores
     parámetros (p, d, q) para la serie.
   • El modelo ARIMA captura la estructura temporal y permite proyecciones
     a corto plazo de valores continuos de PM2.5.

3. CLASIFICACIÓN CON PIPELINES
   ────────────────────────────
   • Se implementaron Pipelines de scikit-learn que integran:
     - Preprocesamiento (imputación, escalado, encoding)
     - Modelos de clasificación
   • El uso de Pipelines garantiza reproducibilidad y evita data leakage.
""")

print(f"""
4. RENDIMIENTO DE MODELOS
   ──────────────────────
   • Mejor modelo: {mejor_modelo_nombre}
   • Accuracy:  {df_resultados.iloc[0]['Accuracy']:.4f}
   • Precision: {df_resultados.iloc[0]['Precision']:.4f}
   • Recall:    {df_resultados.iloc[0]['Recall']:.4f}
   • F1-Score:  {df_resultados.iloc[0]['F1-Score']:.4f}

5. MÉTRICAS DE EVALUACIÓN
   ──────────────────────
   • ACCURACY: Proporción total de predicciones correctas
   • PRECISION: Confiabilidad de las alertas (evitar falsas alarmas)
   • RECALL: Capacidad de detectar todos los casos reales (sensibilidad)
   • Para salud pública, alto RECALL es prioritario para no perder
     episodios de contaminación peligrosa.

6. PREDICCIÓN A 24 HORAS
   ─────────────────────
   • El modelo permite clasificar la calidad del aire esperada
     para cada hora de las próximas 24 horas.
   • Las variables más importantes para la predicción son los
     valores recientes de PM2.5 (lags) y condiciones meteorológicas.
""")

print("="*70)
print("FIN DEL ANÁLISIS")
print("="*70)

---
## **Referencias**

- OMS (2021). WHO global air quality guidelines.
- Hyndman, R.J., & Athanasopoulos, G. (2021). Forecasting: principles and practice.
- Scikit-learn documentation: https://scikit-learn.org/
- Statsmodels documentation: https://www.statsmodels.org/